In [83]:
from urllib.request import urlopen
import pandas as pd
import time
import re
import requests
import os

# 1ツイートに記載する最大試合数
# 1部, 2部, 3部・・・のクラブ名が長いカード対決が重なると許容数を超える可能性があるので念のため4に設定した
MAX_MATCH_INFO = 4

# 最新試合一覧を取得し、検索日付に絞って返却
# プレミアリーグ, FAカップ, Carabaoカップは同時開催されないものとする
def scrapeSportsNaviPage(search_word: str):
    leagues = [
        {'gk': 52, 'name': 'プレミアリーグ'},
        {'gk': 65, 'name': 'FAカップ'},
        {'gk': 598, 'name': 'Carabaoカップ'}
    ]

    for league in leagues:
        url = f'https://soccer.yahoo.co.jp/ws/category/eng/schedule?gk={league["gk"]}'
        html = urlopen(url).read().decode('utf-8')
        df = pd.read_html(html)[0]
        df = df[df['日時'].str.startswith(search_word)]
        print('df', df)

        if not df.empty:
            print(f"DataFrame for {league['name']} is not empty.")
            leagueName = league['name']
            leagueType = df['種別'].values[0] 
            # Your processing logic here
            break  # Exit the loop if DataFrame is not empty

    else:
        print("No data found for the specified search word in any league.")
        
    leagueInfo = leagueName + leagueType
    return df, leagueInfo

In [84]:
import datetime
from types import MappingProxyType
import pandas as pd

# 曜日（英語 ⇒ 日本語）
WEEK_DAYS_DICT: dict = MappingProxyType({
        'Mon':'（月）',
        'Tue':'（火）',
        'Wed':'（水）',
        'Thu':'（木）',
        'Fri':'（金）',
        'Sat':'（土）',
        'Sun':'（日）'
})
    
# 現在の日付と曜日を取得
today = datetime.date.today().strftime("%Y%m%d")
day_of_week = datetime.date.today().strftime("%a")

# 検索キーを作成
ja_day_of_week = WEEK_DAYS_DICT.get(day_of_week)
t_search_key = str(int(today[4:6])) + '/' + str(int(today[6:8])) + ja_day_of_week
print('t_search_key', t_search_key)

t_search_key 1/8（月）


In [85]:
# 検索キーに一致する試合を全件取得
df2, league = scrapeSportsNaviPage(t_search_key)

# スコア列の値が「試合中止」である行を削除
df3 = df2[~df2['スコア'].str.contains('試合中止|試合終了')]

df Empty DataFrame
Columns: [日時, 種別, ホーム, スコア, アウェイ, Unnamed: 5, Unnamed: 6, Unnamed: 7]
Index: []
df                日時                   種別                  ホーム  \
62  1/8（月）  29:15                  3回戦                 ウィガン   
63  1/8（月）  29:15  ※ 試合終了後に結果を掲載いたします。  ※ 試合終了後に結果を掲載いたします。   

                    スコア                 アウェイ           Unnamed: 5  \
62                - 試合前            マンチェスター・U                  NaN   
63  ※ 試合終了後に結果を掲載いたします。  ※ 試合終了後に結果を掲載いたします。  ※ 試合終了後に結果を掲載いたします。   

             Unnamed: 6           Unnamed: 7  
62                  NaN                  NaN  
63  ※ 試合終了後に結果を掲載いたします。  ※ 試合終了後に結果を掲載いたします。  
DataFrame for FAカップ is not empty.
league FAカップ3回戦


In [87]:
# 2023-24プレミアリーグに所属しているクラブのリスト
PREMIER_LEAGUE = [
    "バーンリー",
    "ルートンタウン",
    "チェルシー",
    "フラム",
    "ニューカッスル",
    "マンチェスター・C",
    "エヴァートン",
    "アストン・ヴィラ",
    "マンチェスター・U",
    "トッテナム",
    "アーセナル",
    "クリスタル・パレス",
    "ブレントフォード",
    "フォレスト",
    "シェフィールド・U",
    "ウェストハム",
    "ボーンマス",
    "リヴァプール",
    "ブライトン",
    "ウルヴァーハンプトン",
]


In [93]:
# lambda module設定
# from urllib.request import urlopen
# import pandas as pd
# import time
# import re
# import tweepy
# import modules.settings as Settings
# import requests
# import os

# X-API設定
# client = tweepy.Client(
#     bearer_token = Settings.BEARER_TOKEN,
#     consumer_key = Settings.CONSUMER_KEY,
#     consumer_secret = Settings.CONSUMER_SECRET,
#     access_token = Settings.ACCESS_TOKEN,
#     access_token_secret = Settings.ACCESS_SECRET
# )

In [88]:
# 指定されたメッセージをPostする関数
def _tweetBot(msg : str, parentTweetID: str):
    print('tweetBot start')

    if parentTweetID == '':
        # tweetを実施
        tweet_response = client.create_tweet(
            text=msg
        )
    else:
        # スレッド機能のtweetを実施
        tweet_response = client.create_tweet(
            text=msg,
            in_reply_to_tweet_id=parentTweetID
        )
    print(tweet_response)

    print('tweetBot end')
    return tweet_response.data['id']

In [89]:
# 試合情報からメッセージを作成する関数
def createTweetMsg(df: pd.DataFrame, leagueName: str):
    # トップテキストを用意
    top_text = '今日は' + leagueName + 'が ' + str(len(df)) + ' 試合あるよ！⚽' + '\n' + '絶対に見逃すな！' + '\n\n'
    
    # 変数を初期化
    text = top_text
    tweetID = ''
    match_count = 0
    tweet_count = 0
    match_count_sled = 0
    
    for index, row in df.iterrows():
        match_count += 1
        text = text + (str(row['日時']).split('）')[1].strip() + ' ' + row['ホーム'] + ' VS ' + row['アウェイ']) +  '\n\n'
        
        # マッチ数が5つ以下なら単一ツイートでOK
        if len(df) <= MAX_MATCH_INFO:
            # マッチ数 = match_countとなった場合ツイート
            if len(df) == match_count:
                res = _tweetBot(text, tweetID)
                print('ツイート成功!', res)
            else:
                print('stay...match_count', match_count)
                print('stay...text', text)
        else:
            # 試合カードが5試合たまった場合
            if match_count >= MAX_MATCH_INFO:
                # １回ツイートする
                tweetID = _tweetBot(text, tweetID)
                tweet_count += 1
                # テキストとカウンタを初期化
                text = ''
                match_count = 0
                match_count_sled = len(df) - MAX_MATCH_INFO
            else:
                # スレッド処理を実施
                if match_count_sled == match_count:
                    res = _tweetBot(text, tweetID)
                    print('スレッド処理でツイート成功!', res)
                else:
                    print('stay...match_count', match_count)
                    print('stay...text', text)
    return